In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from src.app.application import Application
import unittest
from src.auth.authentication import AuthenticationModule

from src.app.ui import TkinterUI
from unittest.mock import MagicMock, patch
# from src.app.states.sign_in_up_state import SignInUp
from src.app.app_states import SignInUp

if __name__ == '__main__':
    db = MagicMock()
    app = Application(db=db)
    # overwrite the authentication of the user \"test\" password \"test\"
    with unittest.mock.patch.object(AuthenticationModule, 'authenticate_user', return_value=True):
        initial_state = SignInUp(app)
        ui = TkinterUI(app)
        app._ui = ui
        app.transition_to(initial_state)
        app.run()
   

Application: Transition to SignInUp
Rendering sign in/up page...
Application: Transition to LoggedOut
Rendering logged out page...
Logging in...
User teste authenticated.
Application: Transition to MainState
Rendering main page...
Application: Transition to LoggedOut
Rendering logged out page...
Application: Transition to SignInUp
Rendering sign in/up page...
Application: Transition to SignUpState
Rendering sign up page...
Signing up...
Sign up successful! User rear created.
Application: Transition to LoggedOut
Rendering logged out page...
A janela foi fechada
